In [1]:
import psycopg2
import configparser
import os
import pandas as pd
import time
from nba_api.stats.endpoints import playbyplayv2, leaguegamefinder, boxscoretraditionalv3
from sqlalchemy import create_engine, text
import subprocess
import sys
import papermill as pm
import json
def get_db_config():
    config = configparser.ConfigParser()
    config.read('db.ini')
    
    return {
        'database': config['postgresql']['database'],
        'user': config['postgresql']['user'],
        'password': config['postgresql']['password'],
        'host': config['postgresql']['host'],
        'port': config['postgresql']['port']
    }

def create_table():
    try:      
        # Create the play_by_play_q4 table if it doesn't exist
        create_table_query = """
        CREATE TABLE IF NOT EXISTS play_by_play_q4 (
            game_id VARCHAR(20),
            eventnum INT,
            eventmsgtype INT,
            eventmsgactiontype INT,
            period INT,
            wctimestring VARCHAR(20),
            pctimestring VARCHAR(20),
            homedescription TEXT,
            neutraldescription TEXT,
            visitordescription TEXT,
            score VARCHAR(20),
            scoremargin VARCHAR(10),
            PRIMARY KEY (game_id, eventnum)
        );
        """
        
        cursor.execute(create_table_query)
        conn.commit()
        print("Table play_by_play_q4 created successfully or already exists")

    except Exception as e:
        print(f"Error: {e}")

def alter_table():
    try: 
        alter_table_query = """
        ALTER TABLE play_by_play_q4
        ADD COLUMN IF NOT EXISTS home_players INT[],
        ADD COLUMN IF NOT EXISTS away_players INT[],
        ADD COLUMN IF NOT EXISTS home_3pt_percentage INT[],
        ADD COLUMN IF NOT EXISTS away_3pt_percentage INT[],
        ADD COLUMN IF NOT EXISTS home_freethrow_percentage INT[],
        ADD COLUMN IF NOT EXISTS away_freethrow_percentage INT[];
        """
        cursor.execute(alter_table_query)
        conn.commit()
        print("Table play_by_play_q4 altered successfully")
    except Exception as e:
        print(f"Error: {e}")
        
# connect to database 
db_config = get_db_config()
conn = psycopg2.connect(
    database=db_config['database'],
    user=db_config['user'],
    password=db_config['password'],
    host=db_config['host'],
    port=db_config['port']
)
cursor = conn.cursor()
print("Database connected successfully")


Database connected successfully


In [2]:
# all configurations for nba api requests
CONFIGS = {
    'api_call_sleep' : .600,
    'batch_size' : 1000,
    'seasons' : [
        '2024-25',
        '2023-24',
        '2022-23',
        '2021-22',
        '2020-21',
        '2019-20',
        '2018-19',
        '2017-18',
        '2016-17',
        '2015-16',
        '2014-15',
    ],
    'season_types' : ['Regular Season', 'Playoffs'],
    'min_quarter': 4,
}

NOTEBOOK_PATH = "..\\api_data\\scraping_playbyplay.ipynb"

# Create SQLAlchemy engine for pandas to_sql functionality
CONNECTION_STR = (
    f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}"
    f"@{db_config['host']}:{db_config['port']}/{db_config['database']}"
)
ENGINE = create_engine(CONNECTION_STR)

GAME_IDS =[]

def restart_run_notebook(nb_path=NOTEBOOK_PATH):
    """
    Rerunning the notebook because of API call bottleneck:
    HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
    There are probably better solutions but resetting the kernel was the solution that worked out for me
    """
    print(f"Re-running notebook via Papermill: {nb_path}")
    pm.execute_notebook(
        nb_path,       
        nb_path,           
        log_output=True,
        timeout=None
    )
    print("Notebook run complete.")

def fetch_game_ids():
    # Get game IDs directly from NBA API
    print("Fetching game IDs from NBA API...")
    for season in CONFIGS['seasons']:
        for season_type in CONFIGS['season_types']:
            print(f"Fetching {season} {season_type} games")
            try: 
                gamefinder = leaguegamefinder.LeagueGameFinder(
                    season_nullable=season,
                    season_type_nullable=season_type
                )
                time.sleep(CONFIGS['api_call_sleep'])
                season_games_df = gamefinder.get_data_frames()[0]
                season_game_ids = season_games_df['GAME_ID']
                season_game_ids = season_game_ids[season_game_ids.astype(str).str.startswith(('002','004'))].unique().tolist()
                GAME_IDS.extend(season_game_ids)
                print(f"Found {len(season_game_ids)} games for {season} {season_type}")
            except Exception as e:
                print(f"Error fetching {season} {season_type}: {e}")
        time.sleep(CONFIGS['api_call_sleep'])
    return list(set(GAME_IDS))

def fetch_playbyplay_to_database():
    # Query existing game IDs from the database
    print("Checking for existing games in database...")
    existing_games_query = "SELECT DISTINCT game_id FROM play_by_play_q4"
    existing_games_df = pd.read_sql(existing_games_query, ENGINE)
    existing_game_ids = set(existing_games_df['game_id'].astype(str).tolist() if not existing_games_df.empty else [])
    print(f"Found {len(existing_game_ids)} existing games in database")
    season_patterns = [f"game_id LIKE '0021{i}%%'" for i in range(4, 10)] + [f"game_id LIKE '0022{i}%%'" for i in range(0, 5)] + [f"game_id LIKE '0041{i}%%'" for i in range(4, 10)] + [f"game_id LIKE '0042{i}%%'" for i in range(0, 5)]
    season_conditions = " OR ".join(season_patterns)
    specific_season_query = f"SELECT DISTINCT game_id FROM play_by_play_q4 WHERE {season_conditions}"
    specific_df = pd.read_sql(specific_season_query, ENGINE)
    specific_id = set(specific_df['game_id'].astype(str).tolist() if not specific_df.empty else [])
    print(f"Found {len(specific_id)} existing games from 2014-15 through 2024-25 seasons")

    # Filter for only new game IDs
    new_game_ids = [game_id for game_id in GAME_IDS if game_id not in existing_game_ids]
    print(f"Processing {len(new_game_ids)} new games")
        
    # Fetch and insert play-by-play data for each game
    for i, game_id in enumerate(new_game_ids):
        try:
            print(f"[{i+1}/{len(new_game_ids)}] Fetching for {game_id}")
            pbp = playbyplayv2.PlayByPlayV2(game_id=game_id)
            time.sleep(CONFIGS['api_call_sleep'])
            df = pbp.get_data_frames()[0]
            df = df[df["PERIOD"] >= CONFIGS['min_quarter']]  # Filter for 4th/ot quarter only
                
            if not df.empty:
                # Convert all column names to lowercase to match PostgreSQL default behavior
                df.columns = [col.lower() for col in df.columns]
                    
                # Check which columns from df match our table schema
                cursor.execute("SELECT * FROM play_by_play_q4 LIMIT 0")
                colnames = [desc[0].lower() for desc in cursor.description]
                    
                # Only keep columns that exist in our schema
                df_filtered = df[[col for col in df.columns if col in colnames]]
                    
                # Use if_exists='append' to add to existing table
                df_filtered.to_sql("play_by_play_q4", ENGINE, if_exists="append", index=False, method='multi', chunksize=CONFIGS['batch_size'])
                print(f"Added {len(df_filtered)} plays for game {game_id}")
            else:
                print(f"No 4th quarter data found for game {game_id}")
           
        except Exception as e:
            print(f"Error on {game_id}: {e}")
            if "HTTPSConnectionPool" in str(e):
                print("Detected connection issue, restarting notebook")
                close_db()
                restart_run_notebook(NOTEBOOK_PATH)
                return       
            # adds the game id anyways
            if "duplicate key value violates unique constraint" in str(e):
                with ENGINE.begin() as conn:
                    for _, row in df_filtered.iterrows():
                        row_dict = row.to_dict()
                        columns = row_dict.keys()

                        insert_cols = ", ".join(columns)
                        placeholders = ", ".join([f":{col}" for col in columns])
                        update_cols = [f"{col} = EXCLUDED.{col}" for col in columns if col not in ("game_id", "eventnum")]
                        update_clause = ", ".join(update_cols)

                        query = text(f"""
                            INSERT INTO play_by_play_q4 ({insert_cols})
                            VALUES ({placeholders})
                            ON CONFLICT (game_id, eventnum)
                            DO UPDATE SET {update_clause}
                        """)

                        conn.execute(query, row_dict)
                        print("game updated in database")
    print("Data import completed")

def fetch_players_on_court():
    existing_games_query = """SELECT game_id
                            FROM public.play_by_play_q4
                            WHERE period = 4
                            GROUP BY game_id
                            HAVING COUNT(*) = COUNT(*) FILTER (
                                WHERE home_players IS NULL AND away_players IS NULL
                            );
                            """
    existing_games_df = pd.read_sql(existing_games_query, ENGINE)
    conn = ENGINE.raw_connection()  # Get the connection
    cursor = conn.cursor()          # Get cursor from connection
    for _, game_row in existing_games_df.iterrows():
        try :
            home_sub, away_sub, fourth_quarter_start, fourth_quarter_end = find_substitutions(str(game_row['game_id']))
            home_lineup = get_starting_lineup_fourth(home_sub, fourth_quarter_start)
            away_lineup = get_starting_lineup_fourth(away_sub, fourth_quarter_start)
            print(f"GAME_ID: {game_row['game_id']}")
            print(f"HOME: {home_lineup}")
            print(f"AWAY: {away_lineup}")
            if len(home_lineup) == 5 and len(away_lineup) == 5:
                update_query = """
                        UPDATE play_by_play_q4
                        SET home_players = %s, away_players = %s
                        WHERE game_id = %s AND eventnum = %s 
                        """
                cursor.execute(update_query, (home_lineup, away_lineup, str(game_row['game_id']), int(fourth_quarter_start)))
                conn.commit()
                # go through lineup changes after the first start
                for event_num in range(fourth_quarter_start+1, fourth_quarter_end+1):
                    for player_id, sub_events in home_sub.items():
                        q4 = sub_events.get(4, {'in': [], 'out': []})
                        for event in q4['in']:
                            if event == event_num:
                                home_lineup.append(player_id)
                        for event in q4['out']:
                            if event == event_num:
                                home_lineup.remove(player_id)
                    for player_id, sub_events in away_sub.items():
                        q4 = sub_events.get(4, {'in': [], 'out': []})
                        for event in q4['in']:
                            if event == event_num:
                                away_lineup.append(player_id)
                        for event in q4['out']:
                            if event == event_num:
                                away_lineup.remove(player_id)
                    update_query = """
                        UPDATE play_by_play_q4
                        SET home_players = %s, away_players = %s
                        WHERE game_id = %s AND eventnum = %s
                    """
                    cursor.execute(update_query, (home_lineup, away_lineup, str(game_row['game_id']), int(event_num)))
                    conn.commit()
            else:
                home_lineup = [0,0,0,0,0]
                away_lineup = [0,0,0,0,0]
                update_query = """
                        UPDATE play_by_play_q4
                        SET home_players = %s, away_players = %s
                        WHERE game_id = %s AND eventnum = %s
                        """
                cursor.execute(update_query, (home_lineup, away_lineup, str(game_row['game_id']), int(fourth_quarter_start)))
                conn.commit()
                for event_num in range(fourth_quarter_start+1, fourth_quarter_end+1):
                    update_query = """
                        UPDATE play_by_play_q4
                        SET home_players = %s, away_players = %s
                        WHERE game_id = %s AND eventnum = %s
                    """
                    cursor.execute(update_query, (home_lineup, away_lineup, str(game_row['game_id']), int(event_num)))
                    conn.commit()
                print("starting lineup was not successful")
        except Exception as e:
            if "HTTPSConnectionPool" in str(e):
                print("Detected connection issue, restarting notebook")
                cursor.close()
                conn.close()
                restart_run_notebook(NOTEBOOK_PATH)
                return
    cursor.close()
    conn.close()
    print('data import completed')

def remove_games():
    with ENGINE.begin() as conn:
        conn.execute(text("""
            DELETE FROM play_by_play_q4
            WHERE NOT (
                game_id LIKE '0021%' OR 
                game_id LIKE '0022%' OR 
                game_id LIKE '0041%' OR 
                game_id LIKE '0042%'
            );
        """))
    print("Deleted all games not starting with 0021, 0022, 0041, or 0042")

def find_substitutions(game_id):
    """
    will refactor this and just get 4th quarter and beyond plays only
    """
    pbp = playbyplayv2.PlayByPlayV2(game_id)
    time.sleep(CONFIGS['api_call_sleep'])
    all_plays_df = pbp.get_data_frames()[0]
    all_plays_df = all_plays_df.sort_values('EVENTNUM')
    fourth_quarter = all_plays_df[all_plays_df['PERIOD'] == 4]
    fourth_quarter_start = fourth_quarter.sort_values('EVENTNUM').iloc[0]['EVENTNUM']
    fourth_quarter_end = fourth_quarter['EVENTNUM'].max()
    home_team,away_team = fetch_players_with_minutes(game_id)

    player_subs_home = {}
    player_subs_away = {}
    # adds mapping for all players home/away
    for player_id in home_team:
        player_subs_home[player_id] = {
            4: {'in': [], 'out': []}
        }
    for player_id in away_team:
        player_subs_away[player_id] = {
            4: {'in': [], 'out': []}
        }
    for _, row in fourth_quarter.iterrows():
        if row['EVENTMSGTYPE'] == 8:  # Substitution
            player_out = row['PLAYER1_ID']
            player_in = row['PLAYER2_ID']
            event_num = row['EVENTNUM']
            quarter = row['PERIOD']

            if player_in in home_team:
                team_subs = player_subs_home
            else:
                team_subs = player_subs_away
            # adds the out/in event number data to the players involved in the substitution
            for player_id, direction in [(player_out, 'out'), (player_in, 'in')]:
                team_subs[player_id][quarter][direction].append(event_num)
    return player_subs_home, player_subs_away, fourth_quarter_start, fourth_quarter_end

def get_starting_lineup_fourth(sub_data, event_index):
    """
    Gets the starting lineup through finding substitution timestamps
    """
    starting_lineup = []

    for player_id, quarters in sub_data.items():
        q4 = quarters.get(4, {'in': [], 'out': []})
        print(f"{player_id} : {q4}")
        if len(q4['in']) == 0 and len(q4['out']) == 0:
            starting_lineup.append(player_id)
        elif len(q4['in']) == 0 and len(q4['out']) > 0:
            if q4['out'][0] > event_index:
                starting_lineup.append(player_id)
        elif len(q4['in']) > 0 and len(q4['out']) > 0:
            if q4['out'][0] > event_index and q4['out'][0] < q4['in'][0]:
                starting_lineup.append(player_id)
    return starting_lineup

def fetch_players_with_minutes(game_id):
    """
    Finds all players who played in the 4th quarter. 
    Will need to refactor for overtime periods.
    """
    box = boxscoretraditionalv3.BoxScoreTraditionalV3(
        game_id=game_id,
        start_period=4,
        range_type=1,
        end_period=4
    )
    time.sleep(CONFIGS['api_call_sleep'])
    
    # Extract player stats 
    box_dict = box.get_dict()
    
    all_home_players = box_dict['boxScoreTraditional']['homeTeam']['players']
    all_away_players = box_dict['boxScoreTraditional']['awayTeam']['players']
    
    home_team_players = [
        player['personId'] 
        for player in all_home_players 
        if player.get('statistics', {}).get('minutes')
    ]
    
    away_team_players = [
        player['personId'] 
        for player in all_away_players 
        if player.get('statistics', {}).get('minutes')
    ]
    return home_team_players, away_team_players


def close_db():
    # Close database connections
    if 'cursor' in locals():
        conn.close()
    if 'conn' in locals():
        conn.close()
    print("Database connections closed")
if __name__ == '__main__':
    """GAME_IDS = fetch_game_ids()
    print(len(GAME_IDS))
    fetch_players_on_court()"""
    fetch_players_on_court()

1630174 : {'in': [], 'out': []}
1627783 : {'in': [553], 'out': []}
1626167 : {'in': [523], 'out': [616]}
1629614 : {'in': [522], 'out': []}
1630169 : {'in': [521], 'out': []}
1627741 : {'in': [], 'out': [521]}
1630188 : {'in': [616], 'out': [523, 650]}
1630167 : {'in': [], 'out': [553]}
204456 : {'in': [650], 'out': [522]}
1628415 : {'in': [541], 'out': []}
1631095 : {'in': [], 'out': []}
1630578 : {'in': [493, 652], 'out': [645]}
1630224 : {'in': [542, 624, 651], 'out': [620, 644]}
1641708 : {'in': [492, 644], 'out': [640, 651]}
1628988 : {'in': [620, 640], 'out': [492, 624]}
1641715 : {'in': [], 'out': [541]}
1630256 : {'in': [], 'out': [542]}
201145 : {'in': [645], 'out': [493, 652]}
GAME_ID: 0022300719
HOME: [1630174, 1627741, 1630188, 1630167, 204456]
AWAY: [1631095, 1628988, 1641715, 1630256, 201145]


1630596 : {'in': [], 'out': [591]}
1628386 : {'in': [514], 'out': [610]}
1628378 : {'in': [515], 'out': [592]}
1629636 : {'in': [], 'out': [549]}
1626224 : {'in': [549], 'out': [515]}
1627747 : {'in': [], 'out': []}
201980 : {'in': [591], 'out': [514]}
203526 : {'in': [592], 'out': []}
201577 : {'in': [610], 'out': []}
1629628 : {'in': [570], 'out': [551]}
203944 : {'in': [551], 'out': [615]}
1628973 : {'in': [550], 'out': [611]}
1630167 : {'in': [615], 'out': [550]}
1628404 : {'in': [], 'out': [570]}
1628392 : {'in': [], 'out': []}
1630193 : {'in': [], 'out': []}
1630540 : {'in': [611], 'out': []}
GAME_ID: 0042200132
HOME: [1630596, 1629636, 1626224, 1627747, 201980]
AWAY: [1629628, 1630167, 1628404, 1628392, 1630193]


203114 : {'in': [562], 'out': []}
203507 : {'in': [], 'out': [632]}
203089 : {'in': [544], 'out': []}
203503 : {'in': [543], 'out': []}
202339 : {'in': [563], 'out': [631]}
1628425 : {'in': [631], 'out': [563]}
203953 : {'in': [], 'out': [543]}
203092 : {'in': [], 'out': [544]}
1891 : {'in': [], 'out': [562]}
204098 : {'in': [632], 'out': []}
203501 : {'in': [546], 'out': []}
202498 : {'in': [524], 'out': []}
202683 : {'in': [545], 'out': [578]}
1626144 : {'in': [547], 'out': []}
1628373 : {'in': [], 'out': []}
201563 : {'in': [578], 'out': [524]}
1627786 : {'in': [], 'out': [545]}
203124 : {'in': [], 'out': [546]}
203504 : {'in': [], 'out': [547]}
GAME_ID: 0021700981
HOME: [203507, 1628425, 203953, 203092, 1891]
AWAY: [1628373, 201563, 1627786, 203124, 203504]


2738 : {'in': [], 'out': []}
203110 : {'in': [], 'out': []}
202691 : {'in': [], 'out': []}
201939 : {'in': [378], 'out': []}
203084 : {'in': [396], 'out': []}
2733 : {'in': [], 'out': [378]}
203105 : {'in': [379], 'out': [396]}
201578 : {'in': [], 'out': [379]}
201142 : {'in': [], 'out': []}
201586 : {'in': [], 'out': []}
203500 : {'in': [365], 'out': [395]}
203460 : {'in': [395], 'out': [365]}
201566 : {'in': [], 'out': []}
203079 : {'in': [], 'out': []}
GAME_ID: 0041500317
HOME: [2738, 203110, 202691, 2733, 201578]
AWAY: [201142, 201586, 203460, 201566, 203079]


1629060 : {'in': [], 'out': [589]}
2544 : {'in': [561], 'out': [539]}
203076 : {'in': [539], 'out': []}
1631108 : {'in': [589], 'out': []}
1630559 : {'in': [525], 'out': []}
1627827 : {'in': [], 'out': [561]}
1642261 : {'in': [], 'out': [524]}
1629003 : {'in': [], 'out': [525]}
1629629 : {'in': [524], 'out': []}
1629731 : {'in': [566], 'out': [590]}
1630596 : {'in': [590], 'out': [526]}
1628386 : {'in': [526], 'out': []}
1628378 : {'in': [573], 'out': [541]}
1629636 : {'in': [540], 'out': []}
1629622 : {'in': [541], 'out': []}
1627747 : {'in': [], 'out': [540]}
1627777 : {'in': [], 'out': [573]}
1629660 : {'in': [], 'out': [566]}
GAME_ID: 0022400454
HOME: [1629060, 2544, 1627827, 1642261, 1629003]
AWAY: [1630596, 1628378, 1627747, 1627777, 1629660]


...

Executing: 100%|██████████| 2/2 [6:58:37<00:00, 14774.28s/cell]


Executing: 100%|██████████| 2/2 [6:58:38<00:00, 12559.32s/cell]


Executing: 100%|██████████| 2/2 [7:16:16<00:00, 15397.49s/cell]

Executing: 100%|██████████| 2/2 [7:16:18<00:00, 13089.23s/cell]

Notebook run complete.
